# Disclaimer:
This is just a play-around and visualization notebook. Thus it is not cleaned up.

In [1]:
import sys
sys.path.append("..")

In [2]:
import trimesh
import numpy as np
import json
from src.helper.visualization import visualize_mesh, visualize_meshes, visualize_pointclouds_parts, visualize_pointclouds
import k3d
import sklearn
import networkx as nx
from sklearn.decomposition import PCA
from sklearn.cluster import k_means
import json

In [3]:
mesh = trimesh.load("../data/chair_testmesh.obj")

In [4]:
with open("/home/winter/DL3D-Practical/data/chair_testmesh_offsets.json") as fp:
    offsets = json.load(fp)

face_off = offsets["face_index_offsets"]
vert_off = offsets["vertex_index_offsets"]

face_seq = []
vert_seq = []
for part_n in range(6): 
    face_seq.append(np.arange(face_off[part_n], face_off[part_n+1]+np.random.randint(-3000,0)))
    vert_seq.append(np.arange(vert_off[part_n], vert_off[part_n+1]))

In [5]:
vert_off[-1]

52299

In [7]:
point_clouds = []
for vert in vert_seq:
    n_tot = vert_off[-1]
    n = len(vert)
    mask = np.zeros(mesh.vertices.shape[0], dtype=int)
    mask[vert] = 1
    random_zeros = np.random.randint(0, n_tot, (n//5,))
    random_ones = np.random.randint(0, n_tot, (n//5,))
    mask[random_zeros] = 0
    mask[random_ones] = 1
    point_clouds.append(mesh.vertices[mask.astype(bool),:])

In [8]:
pca = PCA(n_components=3)

In [9]:
def gaussian3d(vec, mu, Sigma):
    return np.exp(-0.5*(vec-mu).T@np.linalg.inv(Sigma)@(vec-mu))

In [10]:
vertices = mesh.vertices
faces = mesh.faces
plot = k3d.plot(name='points', grid_visible=False, grid=(-0.55, -0.55, -0.55, 0.55, 0.55, 0.55))
plt_mesh = k3d.mesh(vertices.astype(np.float32), faces.astype(np.uint32), color=0xd0d0d0)
plot += plt_mesh

n_pc = 0

plt_points = k3d.points((point_clouds[n_pc]).astype(np.float32), point_size=0.005, color=0xff0000)
plot += plt_points


plt_mesh.shader = '3d'
plot.display()

/home/winter/miniconda3/envs/ml3d/lib/python3.10/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float32" does not match required type "float32". A coerced copy has been created.
  warnings.warn(
/home/winter/miniconda3/envs/ml3d/lib/python3.10/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "uint32" does not match required type "uint32". A coerced copy has been created.
  warnings.warn(


Output()

In [20]:
vertices = mesh.vertices
faces = mesh.faces

n_pc = 2
pca.fit(point_clouds[n_pc])
Sigma = pca.get_covariance()
COM = point_clouds[n_pc].mean(axis=0)
opacities = []
for point in vertices:#point_clouds[n_pc]:
    opacities.append(gaussian3d(point,COM, Sigma))


plot = k3d.plot(name='points', grid_visible=False, grid=(-0.55, -0.55, -0.55, 0.55, 0.55, 0.55))
plt_mesh = k3d.mesh(vertices.astype(np.float32), faces.astype(np.uint32), color=0xd0d0d0, attribute=opacities)
plot += plt_mesh

plt_points = k3d.points((point_clouds[n_pc]).astype(np.float32), point_size=0.0025, color=0xffffff, opacity=1)#, attribute=opacities)
plot += plt_points

plot += k3d.points(COM.astype(np.float32), point_size=0.03, color=0xffff00)
o = np.vstack((COM, COM,COM))
plot += k3d.vectors(o, pca.components_)


plt_mesh.shader = '3d'
plot.display()

/home/winter/miniconda3/envs/ml3d/lib/python3.10/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float32" does not match required type "float32". A coerced copy has been created.
  warnings.warn(
/home/winter/miniconda3/envs/ml3d/lib/python3.10/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "uint32" does not match required type "uint32". A coerced copy has been created.
  warnings.warn(
/home/winter/miniconda3/envs/ml3d/lib/python3.10/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(


Output()

In [73]:
o = np.vstack((COM, COM,COM))

In [75]:
o

array([[-0.00050837,  0.06505874, -0.00515511],
       [-0.00050837,  0.06505874, -0.00515511],
       [-0.00050837,  0.06505874, -0.00515511]])

# PCA and COM:

In [60]:
print(point_clouds[2].shape)
COM = point_clouds[2].mean(axis=0)

(15317, 3)


In [61]:
pca = PCA(n_components=3)
pca.fit(point_clouds[2])

PCA(n_components=3)

In [78]:
Sigma = pca.get_covariance()

In [63]:
pca.components_

array([[ 2.32442944e-02,  6.94477544e-01,  7.19138820e-01],
       [ 9.99496455e-01, -6.01790646e-04, -3.17249732e-02],
       [ 2.15995105e-02, -7.19514126e-01,  6.94141832e-01]])

In [138]:
def gaussian3d(vec, mu, Sigma):
    return np.exp(-0.5*(vec-mu).T@np.linalg.inv(Sigma)@(vec-mu))

In [113]:
def sdf_gaussian3d(x, mu, Sigma, zero_val=0.5):
    return gaussian3d(x, mu, Sigma) - zero_val

In [84]:
xyz_range = np.linspace(-0.5,0.5,32)
xx, yy, zz = np.meshgrid(xyz_range, xyz_range, xyz_range)

In [98]:
voxgrid = np.zeros((32,32,32))
for i, x in enumerate(np.linspace(-0.5,0.5,32)):
    for j, y in enumerate(np.linspace(-0.5,0.5,32)):
        for k, z in enumerate(np.linspace(-0.5,0.5,32)):
            gauss = sdf_gaussian3d(x,y,z,COM, Sigma)
            voxgrid[i,j,k] = gauss

In [103]:
voxgrid_scale = np.clip(voxgrid, 0, 255).astype(int)

In [106]:
voxgrid_scale

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 

In [130]:
smaller_pointcloud = []
for point in point_clouds[2]:
    if sdf_gaussian3d(point, COM, Sigma, 3) > 0:
        smaller_pointcloud.append(point)

In [131]:
len(smaller_pointcloud), point_clouds[2].shape

(9213, (15317, 3))

In [136]:
gaussian3d(COM,COM, Sigma)

1.0

In [137]:
gaussian3d(COM+np.array([10,10,10]),COM, Sigma)

0.0

In [163]:
import torch

TrackedArray([[-0.177229  ,  0.495573  ,  0.242403  ],
              [-0.177229  ,  0.497587  ,  0.231531  ],
              [-0.179031  ,  0.466108  ,  0.226189  ],
              ...,
              [ 0.17741005, -0.19883893,  0.15168142],
              [ 0.1754261 , -0.05303471,  0.14976283],
              [ 0.17353941, -0.2755807 ,  0.14641304]])

In [167]:
center_point = torch.mean(point_clouds[n_pc], dim=0) # we use the part's COM
(U, S, V) = torch.pca_lowrank(point_clouds[n_pc])

TypeError: mean() received an invalid combination of arguments - got (TrackedArray, dim=int), but expected one of:
 * (Tensor input, *, torch.dtype dtype)
 * (Tensor input, tuple of ints dim, bool keepdim, *, torch.dtype dtype, Tensor out)
 * (Tensor input, tuple of names dim, bool keepdim, *, torch.dtype dtype, Tensor out)


In [172]:
(U, S, V) = torch.pca_lowrank(torch.tensor(point_clouds[n_pc]))

In [174]:
V.shape, S.shape, U.shape

(torch.Size([3, 3]), torch.Size([3]), torch.Size([14183, 3]))

In [175]:
V.T@torch.diag(S**2)@V

tensor([[187.2323,   3.1620,  59.5361],
        [  3.1620, 482.1296,  -2.2953],
        [ 59.5361,  -2.2953,  95.7977]], dtype=torch.float64)

In [176]:
Sigma

array([[ 1.52721255e-02, -1.85782329e-04, -3.93563794e-05],
       [-1.85782329e-04,  2.82640256e-02,  1.16268947e-02],
       [-3.93563794e-05,  1.16268947e-02,  1.04167193e-02]])

In [178]:
point_clouds[n_pc].shape

(14183, 3)

In [181]:
(point_clouds[n_pc]-COM).T@(point_clouds[n_pc]-COM)/(U.shape[0] - 1)

TrackedArray([[ 1.52721255e-02, -1.85782329e-04, -3.93563794e-05],
              [-1.85782329e-04,  2.82640256e-02,  1.16268947e-02],
              [-3.93563794e-05,  1.16268947e-02,  1.04167193e-02]])

In [180]:
U.shape

torch.Size([14183, 3])

In [183]:
gaussian3d(point_clouds[n_pc],COM, Sigma)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 3 is different from 14183)

In [207]:
def gaussian3d_batched(X, COM, Sigma):
    """
    @param X: shape (N, 3)
    """
    inv_sigma = np.linalg.inv(Sigma)
    X_prime = (X-COM)

    Y = inv_sigma@X_prime.T # (3, N)
    val = np.sum(X_prime.T*Y, axis=0) # (N)

    return np.exp(-0.5*val)

In [208]:
gaussian3d_batched(point_clouds[n_pc], COM, Sigma)

TrackedArray([1.31821649e-01, 1.29070758e-01, 1.63175252e-01, ...,
              2.94395315e-04, 1.33445210e-02, 3.02914557e-05])

In [213]:
gaussian3d(point_clouds[n_pc][2], COM, Sigma)

0.16317525151379178

In [201]:
inv_sigma = np.linalg.inv(Sigma)
X_prime = (X-COM)


In [202]:
Y = inv_sigma@X_prime # (3, N)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 14183 is different from 3)

In [204]:
inv_sigma.shape, X_prime.shape

((3, 3), (14183, 3))

In [23]:
vertices = mesh.vertices
faces = mesh.faces

all_pc = np.concatenate([point_clouds[n_pc] for n_pc in [1,3,4,5]])
pca.fit(all_pc)
Sigma = pca.get_covariance()
COM = all_pc.mean(axis=0)
opacities = []
#for point in all_pc:
#    opacities.append(gaussian3d(point,COM, Sigma))

for point in vertices:#point_clouds[n_pc]:
    opacities.append(gaussian3d(point,COM, Sigma))

plot = k3d.plot(name='points', grid_visible=False, grid=(-0.55, -0.55, -0.55, 0.55, 0.55, 0.55))
plt_mesh = k3d.mesh(vertices.astype(np.float32), faces.astype(np.uint32), color=0xd0d0d0,attribute=opacities)
plot += plt_mesh


plt_points = k3d.points((all_pc).astype(np.float32), point_size=0.00025, color=0xffffff)#, attribute=opacities)
plot += plt_points

plot += k3d.points(COM.astype(np.float32), point_size=0.03, color=0xffff00)
o = np.vstack((COM, COM,COM))
plot += k3d.vectors(o, pca.components_)


plt_mesh.shader = '3d'
plot.display()

/home/winter/miniconda3/envs/ml3d/lib/python3.10/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float32" does not match required type "float32". A coerced copy has been created.
  warnings.warn(
/home/winter/miniconda3/envs/ml3d/lib/python3.10/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "uint32" does not match required type "uint32". A coerced copy has been created.
  warnings.warn(
/home/winter/miniconda3/envs/ml3d/lib/python3.10/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(


Output()

# KMeans:

In [9]:
result = k_means(mesh.vertices, 6)

In [10]:
labels = result[1]

In [11]:
labels.shape, mesh.vertices.shape

((52299,), (52299, 3))

In [12]:
visualize_pointclouds_parts([mesh.vertices], [labels])

/home/winter/miniconda3/envs/playground/lib/python3.9/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float32" does not match required type "float32". A coerced copy has been created.
  warnings.warn(


Output()

In [25]:
for i in range(6):
    mask = labels==i
    np.save(f"../data/mask_{i}.npy", mask)